In [ ]:
#| default_exp markdown.obsidian.personal.machine_learning.tokenize.def_and_notat_token_classification

# markdown.obsidian.personal.machine_learning.tokenize.def_and_notat_token_classification
> Functions for gathering and processing tokenization data and for using ML models trained with such data.

Previous, `trouver` just had functionalities for using ML models to identify newly introduced notations in text and for gathering data to train such models. Moreover, such models were merely classification models, and using these models to identify newly introduced notations had a lot of computational redundancies.

This module aims to provide the same functionalities for both definitions and notations by training and using token classification models instead.

In [ ]:
# TODO: Create a new module dedicated to definition and notation identification and move approparite functions over there. 

In [ ]:
#| export
from collections.abc import Callable
import copy
from itertools import pairwise
import os 
from os import PathLike
from pathlib import Path
import random
from typing import Literal, Optional, TypedDict, Union
import warnings

import bs4
from transformers import BatchEncoding, pipelines, PreTrainedTokenizer, PreTrainedTokenizerFast

from trouver.helper import is_not_space_and_not_punc, split_string_at_indices
from trouver.helper.definition_and_notation import double_asterisk_indices, notation_asterisk_indices
from trouver.helper.html import (
    add_HTML_tag_data_to_raw_text, add_space_to_lt_symbols_without_space, remove_html_tags_in_text, StrAndHTMLTagsWithIndices,
    HTMLTagWithIndices)
from trouver.helper.latex import (
    _is_balanced_braces, _first_curly_bracket, _last_curly_bracket, random_char_modification,
    dollar_sign_manipulation, remove_math_keywords, random_word_removal, random_latex_command_removal,
    push_dollar_signs, augment_text,
    change_font_styles_at_random, change_greek_letters_at_random, remove_font_styles_at_random
    ) 
from trouver.helper.regex import latex_indices, replace_string_by_indices
from trouver.markdown.markdown.file import MarkdownFile, MarkdownLineEnum
from trouver.markdown.obsidian.personal.note_processing import process_standard_information_note
from trouver.markdown.obsidian.vault import VaultNote




In [ ]:
from unittest import mock
import shutil
import tempfile

from datasets import ClassLabel, Dataset, Features, Sequence, Value
from transformers import AutoTokenizer
from fastcore.test import *

from trouver.helper.tests import _test_directory


## Gather ML data from information notes

In [ ]:
#| export
def convert_double_asterisks_to_html_tags(
        text: str
        ) -> str:
    """
    Replace the double asterisks, which signify definitions and notations,
    in `text` with HTML tags.
    """
    double_asts = double_asterisk_indices(text)
    replacement_html_tags = [
        _html_tag_from_double_ast(text[start:end])
        for start, end in double_asts]
    return replace_string_by_indices(
        text, double_asts, replacement_html_tags)


def _html_tag_from_double_ast(
        double_ast_string: str # Starts and ends with double asts
        ) -> str:
    """
    Get the HTML tag representing definition or notation data from
    a string surrounded by double asterisks.

    This is used in the `_convert_double_asterisks_to_html_tags` function.
    """
    no_asts = double_ast_string[2:-2]
    if notation_asterisk_indices(double_ast_string):
        return f'<span notation="">{no_asts}</span>'
    else:
        return f'<b definition="">{no_asts}</b>'


In [ ]:
print(convert_double_asterisks_to_html_tags("**hi**. Here is a notation **$asdf$**"))
test_eq(convert_double_asterisks_to_html_tags("**hi**. Here is a notation **$asdf$**"), '<b definition="">hi</b>. Here is a notation <span notation="">$asdf$</span>')


<b definition="">hi</b>. Here is a notation <span notation="">$asdf$</span>


In [ ]:
#| export
def raw_text_with_html_tags_from_markdownfile(
        mf: MarkdownFile,
        vault: PathLike
        ) -> str:
    """
    Process the `MarkdownFile`, replacing the double asterisk surrounded
    text indicating definitions and notations to be HTML tags instead.
    """
    mf = process_standard_information_note(
        mf, vault, remove_double_asterisks=False,
        remove_html_tags=False)
    return convert_double_asterisks_to_html_tags(str(mf))



In [ ]:
#| hide

# TODO: 
# I want to make sure that footnotes are getting properly removed.
mf = MarkdownFile.from_string(
    r"""---
aliases: []
tags: []
---
# Something  

Some kind of potato[^2]

[^2]: Some footnote

[[link_to_note|Some link]]


# See Also
# Meta
## References and Citations
""") 
raw_text_with_html_tags_from_markdownfile(mf, None)


'Some kind of potato[^2]\n\n[^2]: Some footnote\n\nSome link\n'

In [ ]:
#| hide
mf = MarkdownFile.from_string(
    r"""---
aliases: []
tags: []
---
# Galois group of a separable and normal finite field extension

Let $L/K$ be a separable and normal finite field extension. Its <b definition="Galois group of a separable and normal finite field extension">Galois group</b> <span notation="">$\operatorname{Gal}(L/K)$</span> is...

# Galois group of a separable and normal profinite field extension

In fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that
$L = \varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its **Galois group** **$\operatorname{Gal}(L/K)$**

# See Also
# Meta
## References and Citations
""")

In the following example, let `mf` be the following `MarkdownFile`:

In [ ]:
print(str(mf))

---
aliases: []
tags: []
---
# Galois group of a separable and normal finite field extension

Let $L/K$ be a separable and normal finite field extension. Its <b definition="Galois group of a separable and normal finite field extension">Galois group</b> <span notation="">$\operatorname{Gal}(L/K)$</span> is...

# Galois group of a separable and normal profinite field extension

In fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that
$L = \varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its **Galois group** **$\operatorname{Gal}(L/K)$**

# See Also
# Meta
## References and Citations


The `raw_text_with_html_tags_from_markdownfile` function processes the `MarkdownFile` much in the same way as the `process_standard_information_note` function, except it 1. preserves HTML tags, and 2. replaces text surrounded by double asterisks `**` with HTML tags signifiying whether the text displays a definition or a notation.

In the below example, note that the `vault` parameter is set to `None`; this is fine for this example becaues the `process_standard_information_note` function only needs a `vault` argument when embedded links need to be replaced with text (via the `MarkdownFile.replace_embedded_links_with_text` function), but `mf` has no embedded links.

In [ ]:
print(raw_text_with_html_tags_from_markdownfile(mf, None))

Let $L/K$ be a separable and normal finite field extension. Its <b definition="Galois group of a separable and normal finite field extension">Galois group</b> <span notation="">$\operatorname{Gal}(L/K)$</span> is...

In fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that
$L = \varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its <b definition="">Galois group</b> <span notation="">$\operatorname{Gal}(L/K)$</span>



In [ ]:
#| hide
assert '**' not in raw_text_with_html_tags_from_markdownfile(mf, None)

In [ ]:
#| export
class HTMLData(TypedDict):
    note_name: str
    raw_text: str
    tags: list[HTMLTagWithIndices]
    # list[bs4.element.Tag]


In [ ]:
#| export
def html_data_from_note(
        note_or_mf: Union[VaultNote, MarkdownFile], # Either a `VaultNote`` object to a note or a `MarkdownFile` object from which to extra html data.
        vault: Optional[PathLike] = None, # If vault to use when processing the `MarkdownFile` objects (if `note_of_mf` is a `VaultNote`, then this `MarkdownFile` object is created from the text of the note), cf. the `process_standard_information_note` function.
        note_name: Optional[str] = None, # If `note_or_mf` is a `MarkdownFile`, `note_name` should be the name of the note from which the `MarkdownFile` comes from if applicable. If `note_or_mf` is a `VaultNote` object, then `note_name` is ignored and `note_or_mf.name` is used instead.
        ) -> Union[HTMLData, None]: # The keys to the dict are "note_name", "raw_text", "tags". However, `None` is returned if `note` does not exist or the note is marked with auto-generated, unverified data.
    # TODO: implement obtaining multiple datapoints from a single note
    # Via typos for example.
    # TODO: implement various data augmentation techniques
    """Obtain html data for token classification from the information note.

    Currently, the token types mainly revolve around definitions and
    notations.

    If `note` has the tag `_auto/def_and_notat_identified`, then the data
    in the note is assumed to be auto-generated and not verified and
    `None` is returned.

    **Returns**
    - Union[dict, None]
        - The keys-value pairs are 
            - `"note_name"` - The name of the note
            - `"raw_text"` - The raw text to include in the data.
            - `"tags"` - The list with HTML tags carrying definition/notation
              data and their locations in the Raw text. See the second output to
              the function `remove_html_tags_in_text`.
                - Each element of the list is a tuple consisting of a ``bs4.element.Tag``
                  and two ints.
    """
    if isinstance(note_or_mf, VaultNote) and not note_or_mf.exists():
        return None
    if isinstance(note_or_mf, VaultNote):
        mf = MarkdownFile.from_vault_note(note_or_mf)
        note_name = note_or_mf.name
        if vault is None:
            vault = note_or_mf.vault
    else: # isinstance(note_or_mf, MarkdownFile):
        mf = note_or_mf.copy(deep=False)
    if mf.has_tag('_auto/def_and_notat_identified'):
        return None
    raw_text_with_tags = raw_text_with_html_tags_from_markdownfile(mf, vault)
    raw_text, tags_and_locations = remove_html_tags_in_text(raw_text_with_tags)

    return HTMLData(note_name=note_name, raw_text=raw_text, tags=tags_and_locations)

In the following example, we mock a `VaultNote` whose content is that of `mf` in the example for the `raw_text_with_html_tags_from_markdownfile` function. Note that there is some text surrounded by double within `mf` surrounded by double asterisks `**` and some text surrounded by HTML tags to indicate definitions and notations introduced.

In [ ]:
mf = MarkdownFile.from_string(
    r"""---
aliases: []
tags: []
---
# Galois group of a separable and normal finite field extension

Let $L/K$ be a separable and normal finite field extension. Its <b definition="Galois group of a separable and normal finite field extension">Galois group</b> <span notation="">$\operatorname{Gal}(L/K)$</span> is...

# Galois group of a separable and normal profinite field extension

In fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that
$L = \varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its **Galois group** **$\operatorname{Gal}(L/K)$**

# See Also
# Meta
## References and Citations
""")

with (mock.patch('__main__.VaultNote') as mock_VaultNote,
      mock.patch('__main__.MarkdownFile.from_vault_note') as mock_from_vault_note,
      mock.patch('__main__.isinstance') as mock_isinstance):
    mock_VaultNote.exists.return_value = True
    mock_VaultNote.name = "Note's name"
    mock_from_vault_note.return_value = mf
    mock_isinstance.return_value = True

    print(f"The following is the text from mf:\n\n{str(mf)}")

    html_data = html_data_from_note(mock_VaultNote, None)
    print(html_data)

    test_eq(html_data['note_name'], "Note's name")
    assert '**' not in html_data['raw_text']
    assert '<' not in html_data['raw_text']  # Test the lack of HTML tags in the raw text

    print(html_data['tags'])
    test_eq(len(html_data['tags']), 4)
    assert isinstance(html_data['tags'][0][0], bs4.element.Tag)
    assert html_data['tags'][0][0].has_attr('definition')
    assert not html_data['tags'][0][0].has_attr('notation')
    assert html_data['tags'][1][0].has_attr('notation')
    assert not html_data['tags'][1][0].has_attr('definition')
    assert html_data['tags'][2][0].has_attr('definition')
    assert not html_data['tags'][2][0].has_attr('notation')
    assert html_data['tags'][3][0].has_attr('notation')
    assert not html_data['tags'][3][0].has_attr('definition')

The following is the text from mf:

---
aliases: []
tags: []
---
# Galois group of a separable and normal finite field extension

Let $L/K$ be a separable and normal finite field extension. Its <b definition="Galois group of a separable and normal finite field extension">Galois group</b> <span notation="">$\operatorname{Gal}(L/K)$</span> is...

# Galois group of a separable and normal profinite field extension

In fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that
$L = \varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its **Galois group** **$\operatorname{Gal}(L/K)$**

# See Also
# Meta
## References and Citations
{'note_name': "Note's name", 'raw_text': 'Let $L/K$ be a separable and normal finite field extension. Its Galois group $\\operatorname{Gal}(L/K)$ is...\n\nIn fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal pr

We can also just pass a `MarkdonwFile` object instead of a `VaultNote` object. In this case, we can specify the `note_name` parameter to indicate which note the `MarkdownFile` object came from, if applicable.

In [ ]:
html_data = html_data_from_note(mf, vault=None, note_name="Note's name")
print(html_data)

test_eq(html_data['note_name'], "Note's name")
assert '**' not in html_data['raw_text']
assert '<' not in html_data['raw_text']  # Test the lack of HTML tags in the raw text

print(html_data['tags'])
test_eq(len(html_data['tags']), 4)
assert isinstance(html_data['tags'][0][0], bs4.element.Tag)
assert html_data['tags'][0][0].has_attr('definition')
assert not html_data['tags'][0][0].has_attr('notation')
assert html_data['tags'][1][0].has_attr('notation')
assert not html_data['tags'][1][0].has_attr('definition')
assert html_data['tags'][2][0].has_attr('definition')
assert not html_data['tags'][2][0].has_attr('notation')
assert html_data['tags'][3][0].has_attr('notation')
assert not html_data['tags'][3][0].has_attr('definition')

{'note_name': "Note's name", 'raw_text': 'Let $L/K$ be a separable and normal finite field extension. Its Galois group $\\operatorname{Gal}(L/K)$ is...\n\nIn fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that\n$L = \\varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its Galois group $\\operatorname{Gal}(L/K)$\n', 'tags': [HTMLTagWithIndices(tag=<b definition="Galois group of a separable and normal finite field extension">Galois group</b>, start=64, end=76), HTMLTagWithIndices(tag=<span notation="">$\operatorname{Gal}(L/K)$</span>, start=77, end=102), HTMLTagWithIndices(tag=<b definition="">Galois group</b>, start=330, end=342), HTMLTagWithIndices(tag=<span notation="">$\operatorname{Gal}(L/K)$</span>, start=343, end=368)]}
[HTMLTagWithIndices(tag=<b definition="Galois group of a separable and normal finite field extension">Galois group</b>, start=64, end=76), HTMLTagWithIndices(tag

If we do not specify `note_name`, then `None` is used for the `'Note name'` key in the output:

In [ ]:
html_data = html_data_from_note(mf, vault=None, note_name=None)
print(html_data)

assert html_data['note_name'] is None

{'note_name': None, 'raw_text': 'Let $L/K$ be a separable and normal finite field extension. Its Galois group $\\operatorname{Gal}(L/K)$ is...\n\nIn fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that\n$L = \\varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its Galois group $\\operatorname{Gal}(L/K)$\n', 'tags': [HTMLTagWithIndices(tag=<b definition="Galois group of a separable and normal finite field extension">Galois group</b>, start=64, end=76), HTMLTagWithIndices(tag=<span notation="">$\operatorname{Gal}(L/K)$</span>, start=77, end=102), HTMLTagWithIndices(tag=<b definition="">Galois group</b>, start=330, end=342), HTMLTagWithIndices(tag=<span notation="">$\operatorname{Gal}(L/K)$</span>, start=343, end=368)]}


For the following example, the note has an HTML tag already with extra data (attributes other than `'definition'` or `'notation'`). We assert that the extra data is preserved. 

In [ ]:
with (mock.patch('__main__.VaultNote') as mock_VaultNote,
      mock.patch('__main__.MarkdownFile.from_vault_note') as mock_from_vault_note,
      mock.patch('__main__.isinstance') as mock_isinstance):
    mock_VaultNote.exists.return_value = True
    mock_VaultNote.name = "Note's name"
    mock_isinstance.return_value = True

    text = r'Let $X$ be a topological space and let $U \subseteq X$ be an subspace. The <b definition="Closure of a subspace of a topological space" typo="dosure of $U$">closure of $U$</b> is defined as...'
    mf = MarkdownFile.from_string(text)
    mock_from_vault_note.return_value = mf
    print(f"The following is the text of the mocked note: \n\n {text}\n\n")

    html_data = html_data_from_note(mock_VaultNote, None)
    print(html_data)
    assert html_data['tags'][0][0].has_attr('typo')
    test_eq(html_data['tags'][0][0].attrs['typo'], 'dosure of $U$')

The following is the text of the mocked note: 

 Let $X$ be a topological space and let $U \subseteq X$ be an subspace. The <b definition="Closure of a subspace of a topological space" typo="dosure of $U$">closure of $U$</b> is defined as...


{'note_name': "Note's name", 'raw_text': 'Let $X$ be a topological space and let $U \\subseteq X$ be an subspace. The closure of $U$ is defined as...', 'tags': [HTMLTagWithIndices(tag=<b definition="Closure of a subspace of a topological space" typo="dosure of $U$">closure of $U$</b>, start=75, end=89)]}


In the following example, the (mocked) note has the `#_auto/def_and_notats_identified` tag to indicate that its definition and notation markings were auto-generated by a model (trained with data processed by the `tokenize_html_data` function) using the `auto_mark_def_and_notats` function. In this case, the `html_data_from_note` function returns `None` to prevent gathering data that is unverified and auto-generated by a model.

In [ ]:
# with (mock.patch('__main__.VaultNote') as mock_VaultNote,
#       mock.patch('__main__.MarkdownFile.from_vault_note') as mock_from_vault_note):
#     mock_VaultNote.exists.return_value = True
#     mock_VaultNote.name = "Note's name"
text = r'''---
tags: [_auto/def_and_notat_identified]
---
Let $X$ be a topological space and let $U \subseteq X$ be an subspace. The <b definition="Closure of a subspace of a topological space" typo="dosure of $U$">closure of $U$</b> is defined as...'''

mf = MarkdownFile.from_string(text)
mock_from_vault_note.return_value = mf
print(f"The following is the text of the mocked note: \n\n{text}\n\n")

html_data = html_data_from_note(note_or_mf=mf)
assert(html_data is None)

The following is the text of the mocked note: 

---
tags: [_auto/def_and_notat_identified]
---
Let $X$ be a topological space and let $U \subseteq X$ be an subspace. The <b definition="Closure of a subspace of a topological space" typo="dosure of $U$">closure of $U$</b> is defined as...




In [ ]:
#| export
def tokenize_html_data(
        html_locus: HTMLData, # An output of `html_data_from_note`
        tokenizer: Union[PreTrainedTokenizer, PreTrainedTokenizerFast],
        max_length: int, # Max length for each sequence of tokens
        ner_tag_from_html_tag: Callable[[bs4.element.Tag], Union[str, 'None']], # takes in a bs4.element.Tag and outputs the ner_tag (as a string or `None`)
        label2id: dict[str, int], # The keys are ner_tag's of the form f"I-{output}" or f"B-{output}" where `output` is an output of `ner_tag_from_html_tag`.
        default_label: str = "O", # The default label for the NER tagging.
        ) -> tuple[list[list[str]], list[list[int]]]: # The first list consists of the tokens and the second list consists of the named entity recognition tags.
    """Actually tokenize the html data outputted by `html_data_from_note`.

    To account for the possibility that the raw text is long,
    this function uses the `tokenizer.batch_encode_plus` function
    to tokenize the text into sequences. 
    """
    tokenized = tokenizer.batch_encode_plus(
        [html_locus["raw_text"]], max_length=max_length, return_overflowing_tokens=True,
        return_offsets_mapping=True, truncation=True)

    default_id = label2id[default_label]        
    ner_ids = [[default_id for _ in seq_input_ids]
               for seq_input_ids in tokenized['input_ids']]
    for tag, start, end in html_locus['tags']:
        ner_tag = ner_tag_from_html_tag(tag)
        if ner_tag is None:
            continue  # `ner_tag` is not of relevant data.
        tuppy = _start_end_seqs_indices_for_html_tag(tokenized, start, end - 1)
        (start_seq, start_index_in_seq), (end_seq, end_index_in_seq) = tuppy
        _set_ner_ids_for_tag(
            ner_ids, start_seq, start_index_in_seq, end_seq, end_index_in_seq,
            label2id, ner_tag)
    # return tokenized["input_ids"], ner_ids
    tokens = [tokenizer.convert_ids_to_tokens(tokens_for_seq)
              for tokens_for_seq in tokenized["input_ids"]]
    return tokens, ner_ids


def _start_end_seqs_indices_for_html_tag(
        tokenized: BatchEncoding,
        tag_start_ind: int,
        tag_end_ind: int
        ) -> tuple[tuple[int, int], tuple[int, int]]: # The first tuple is `(a, b)` where `tokenized['input_ids'][a][b]` is the token corresponding to the start of the HTML tag's (raw) text. The second tuple is `(c, d)` where `tokenized['input_ids'][c][d]` is the token corresponding to the end of the HTML tag's (raw) text.
    start_seq = _search_seq_ind_for_char(tokenized['offset_mapping'], tag_start_ind)
    # start_index_in_seq = tokenized.char_to_token(batch_or_char_index=start_seq, char_index=tag_start_ind)
    start_index_in_seq = _search_within_seq_for_char(tokenized['offset_mapping'][start_seq], tag_start_ind)
    end_seq = _search_seq_ind_for_char(tokenized['offset_mapping'], tag_end_ind)
    # end_index_in_seq = tokenized.char_to_token(batch_or_char_index=end_seq, char_index=tag_end_ind)
    end_index_in_seq = _search_within_seq_for_char(tokenized['offset_mapping'][end_seq], tag_end_ind)
    return (start_seq, start_index_in_seq), (end_seq, end_index_in_seq)


def _min_max_char_ind_for_seq(
        offset_for_seq: list[tuple[int,int]] # An item in tokenized['offset_mapping']
        ):
    min_char_ind, max_char_ind = 0, 0
    for inds in offset_for_seq:
        if inds != (0,0):
            min_char_ind = inds[0]
            break
    for inds in reversed(offset_for_seq):
        if inds != (0,0):
            max_char_ind = inds[1]
            break
    return min_char_ind, max_char_ind

def _char_is_in_seq(
        offset_for_seq: list[int], # An item in tokenized['offset_mapping']
        char: int # The index of a character in the original raw text
        ) -> bool:
    min_char_ind, max_char_ind = _min_max_char_ind_for_seq(offset_for_seq)
    return min_char_ind <= char and char < max_char_ind

def _search_seq_ind_for_char(
        offsets: list[tuple[int, int]], # tokenized['offset_mapping']
        char: int # The index of a character in the original raw text
        ) -> int:
    """
    Binary search the index of the sequence containing the token at the 
    location of the index `char` within the original (raw) text.

    Based on pseudocode from https://pseudoeditor.com/guides/binary-search
    """
    left = 0
    right = len(offsets) - 1
    while left <= right:
        mid = (left + right) // 2
        min_char_ind, max_char_ind = _min_max_char_ind_for_seq(offsets[mid])
        if min_char_ind <= char and char < max_char_ind:
            return mid
        elif max_char_ind <= char:
            left = mid + 1
        else:
            right = mid - 1
    return -1  # This should not be returned under normal use.


def _search_within_seq_for_char(
        seq_offset: list[tuple[int, int]],
        char: int
    ) -> int:
    """
    Binary search for the index within the sequence corresponding
    to the token at the location of the index `char` within the
    original (raw) text.

    Based on pseudocode from https://pseudoeditor.com/guides/binary-search
    """
    left = 0
    right = len(seq_offset) - 1
    while left <= right:
        mid = (left + right) // 2
        min_char_ind, max_char_ind = seq_offset[mid] 
        if min_char_ind <= char and char < max_char_ind:
            return mid
        elif max_char_ind <= char:
            left = mid + 1
        else:
            right = mid - 1
    return -1  # This should not be returned under normal use.


def _set_ner_ids_for_tag(
        ner_ids: list[list[int]],
        start_seq: int, 
        start_index_in_seq: int,
        end_seq: int,
        end_index_in_seq: int,
        label2id: dict[str, int],
        ner_tag: str
        ) -> None:
    """
    After the locations of the tokens corresponding to a HTML tag have been found, 
    mark within `ner_ids` the appropriate NER tags at the locations corresponding
    to the tokens' locations.
    """
    ner_ids[start_seq][start_index_in_seq] = label2id[f"B-{ner_tag}"]
    i_ner_id = label2id[f"I-{ner_tag}"]
    seq, ind = start_seq, start_index_in_seq + 1
    while seq < end_seq or ind <= end_index_in_seq:
        if len(ner_ids[seq]) <= ind:
            seq += 1
            ind = 0
        else:
            ner_ids[seq][ind] = i_ner_id 
            ind += 1
    


def def_or_notat_from_html_tag(
        tag: bs4.element.Tag
        ) -> Union[str, None]:
    """
    Can be passed as the `ner_tag_from_html_tag` argument in `tokenize_html_data`
    for the purposes of compiling a dataset for definition and notation
    identification.

    The strings f"I-{output}" and f"B-{output}" are valid ner_tags. To use for 
    """
    if "definition" in tag.attrs:
        return "definition"
    elif "notation" in tag.attrs:
        return "notation"
    return None  # If the HTML tag carries neither definition nor notation data.

In [ ]:
#| hide
test_eq(_min_max_char_ind_for_seq([(0,0), (1,3), (3,4), (4,7), (7,15), (0,0)]), (1,15))

offsets = [[(0,0), (0,3), (4,5), (5,6), (6,7), (7,8), (8,9),],
           [(10,12), (13,14), (15,18), (18,24)],
           [(25,28), (29,35), (36,42), ]]
test_eq(_search_seq_ind_for_char(offsets, 0), 0)
test_eq(_search_seq_ind_for_char(offsets, 1), 0)
test_eq(_search_seq_ind_for_char(offsets, 5), 0)
test_eq(_search_seq_ind_for_char(offsets, 8), 0)
# I don't think that character index 9 is something that I need to worry about.
test_eq(_search_seq_ind_for_char(offsets, 10), 1)
test_eq(_search_seq_ind_for_char(offsets, 23), 1)
test_eq(_search_seq_ind_for_char(offsets, 25), 2)
test_eq(_search_seq_ind_for_char(offsets, 41), 2)

We continue with an example using the HTML data from the example for the `html_data_from_note` function.

In [ ]:
mf = MarkdownFile.from_string(
    r"""---
aliases: []
tags: []
---
# Galois group of a separable and normal finite field extension

Let $L/K$ be a separable and normal finite field extension. Its <b definition="Galois group of a separable and normal finite field extension">Galois group</b> <span notation="">$\operatorname{Gal}(L/K)$</span> is...

# Galois group of a separable and normal profinite field extension

In fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that
$L = \varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its **Galois group** **$\operatorname{Gal}(L/K)$**

# See Also
# Meta
## References and Citations
""")

html_data = html_data_from_note(mf, vault=None, note_name=None)
print(html_data)

assert html_data['note_name'] is None

{'note_name': None, 'raw_text': 'Let $L/K$ be a separable and normal finite field extension. Its Galois group $\\operatorname{Gal}(L/K)$ is...\n\nIn fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that\n$L = \\varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its Galois group $\\operatorname{Gal}(L/K)$\n', 'tags': [HTMLTagWithIndices(tag=<b definition="Galois group of a separable and normal finite field extension">Galois group</b>, start=64, end=76), HTMLTagWithIndices(tag=<span notation="">$\operatorname{Gal}(L/K)$</span>, start=77, end=102), HTMLTagWithIndices(tag=<b definition="">Galois group</b>, start=330, end=342), HTMLTagWithIndices(tag=<span notation="">$\operatorname{Gal}(L/K)$</span>, start=343, end=368)]}


In [ ]:
html_data['raw_text']
html_data["tags"]

[HTMLTagWithIndices(tag=<b definition="Galois group of a separable and normal finite field extension">Galois group</b>, start=64, end=76),
 HTMLTagWithIndices(tag=<span notation="">$\operatorname{Gal}(L/K)$</span>, start=77, end=102),
 HTMLTagWithIndices(tag=<b definition="">Galois group</b>, start=330, end=342),
 HTMLTagWithIndices(tag=<span notation="">$\operatorname{Gal}(L/K)$</span>, start=343, end=368)]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
label2id = {
    "O": 0,
    "B-definition": 1,
    "I-definition": 2,
    "B-notation": 3,
    "I-notation": 4
}
tokens, ner_tag_ids = tokenize_html_data(html_data, tokenizer, 510, def_or_notat_from_html_tag, label2id)

For this example, `max_length` is set to 510 (tokens). The string ("Raw text") is not very long, so only one sequence should be present.

In [ ]:
test_eq(len(tokens), 1)
test_eq(len(ner_tag_ids), 1)

Now let us see what has been tagged:

In [ ]:
id2label = {value: key for key, value in label2id.items()}
id2label

{0: 'O',
 1: 'B-definition',
 2: 'I-definition',
 3: 'B-notation',
 4: 'I-notation'}

In [ ]:
for token, ner_tag in zip(tokens[0], ner_tag_ids[0]):
    if ner_tag != 0:
        print(f"{token}\t\t{id2label[ner_tag]}")

gal		B-definition
##ois		I-definition
group		I-definition
$		B-notation
\		I-notation
operator		I-notation
##name		I-notation
{		I-notation
gal		I-notation
}		I-notation
(		I-notation
l		I-notation
/		I-notation
k		I-notation
)		I-notation
$		I-notation
gal		B-definition
##ois		I-definition
group		I-definition
$		B-notation
\		I-notation
operator		I-notation
##name		I-notation
{		I-notation
gal		I-notation
}		I-notation
(		I-notation
l		I-notation
/		I-notation
k		I-notation
)		I-notation
$		I-notation


Let us set `max_length` to be shorter to observe an example of a tokenization of a single text across multiple sequences (Of course, in practice, the max token length would be set to be longer, say around 512 or 1024.):

In [ ]:
token_ids, ner_tag_ids = tokenize_html_data(html_data, tokenizer, 20, def_or_notat_from_html_tag, label2id)

In [ ]:
print(len(token_ids))
print(len(ner_tag_ids))

7
7


In [ ]:
ner_tag_ids

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2],
 [2, 2, 2, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 3, 4, 4],
 [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0]]

## Augmenting data

In [ ]:
#| export
def _split_text_by_html_data_parts(
        # text_tags_and_locations = StrAndHTMLTagsWithIndices
        datapoint: HTMLData
        ) -> list[tuple[str, Union[bs4.element.Tag, None]]]:
    r"""
    Helper function to `augment_html_data`.
    """
    to_return: list[tuple[str, Union[bs4.element.Tag, None]]] = []
    split_points: list[int] = []
    for tag_ind in datapoint['tags']:
        split_points.append(tag_ind.start)
        split_points.append(tag_ind.end)
    pieces: list[str] = split_string_at_indices(datapoint['raw_text'], split_points)
    for i, piece in enumerate(pieces):
        if i % 2 == 0:
            to_return.append((piece, None))
        else:
            to_return.append((piece, datapoint['tags'][int(i/2)].tag))
    return to_return


In [ ]:
#|hide
sample_text = r"""# Galois group of a separable and normal finite field extension

Let $L/K$ be a separable and normal finite field extension. Its <b definition="Galois group of a separable and normal finite field extension">Galois group</b> <span notation="">$\operatorname{Gal}(L/K)$</span> is...

# See Also
# Meta
## References and Citations
"""

sample_str_and_html_tags_with_indices: StrAndHTMLTagsWithIndices = remove_html_tags_in_text(sample_text)
sample_html_data = HTMLData(note_name=None, raw_text=sample_str_and_html_tags_with_indices.raw_text, tags=sample_str_and_html_tags_with_indices.tags)
output = _split_text_by_html_data_parts(sample_html_data)
assert isinstance(output[1][1], bs4.element.Tag)
output


[('# Galois group of a separable and normal finite field extension\n\nLet $L/K$ be a separable and normal finite field extension. Its ',
  None),
 ('Galois group',
  <b definition="Galois group of a separable and normal finite field extension">Galois group</b>),
 (' ', None),
 ('$\\operatorname{Gal}(L/K)$',
  <span notation="">$\operatorname{Gal}(L/K)$</span>),
 (' is...\n\n# See Also\n# Meta\n## References and Citations\n', None)]

In [ ]:
#| hide
sample_text = r"""<b definition="">Hello</b> asdf <b notation=""> $\operatorname{Gal}$</b>"""
sample_str_and_html_tags_with_indices: StrAndHTMLTagsWithIndices = remove_html_tags_in_text(sample_text)
sample_html_data = HTMLData(note_name=None, raw_text=sample_str_and_html_tags_with_indices.raw_text, tags=sample_str_and_html_tags_with_indices.tags)
output = _split_text_by_html_data_parts(sample_html_data)
assert isinstance(output[1][1], bs4.element.Tag)
assert isinstance(output[3][1], bs4.element.Tag)
test_is(output[4][1], None)
output

[('', None),
 ('Hello', <b definition="">Hello</b>),
 (' asdf ', None),
 (' $\\operatorname{Gal}$', <b notation=""> $\operatorname{Gal}$</b>),
 ('', None)]

In [ ]:
print(sample_str_and_html_tags_with_indices.tags)

[HTMLTagWithIndices(tag=<b definition="">Hello</b>, start=0, end=5), HTMLTagWithIndices(tag=<b notation=""> $\operatorname{Gal}$</b>, start=11, end=32)]


In [ ]:
#| export
def augment_html_data(
        datapoint: HTMLData,
        num_augmentation_sets: int = 1, # Each augmentation set consists of an augmentation with low, medium, and high probability modifications.
        seed: Optional[int] = None
        ) -> list[HTMLData]:
    r"""Augment a given datapoint for HTML tagging.

    """
    augmented_datapoints: list[HTMLData] = []
    pieces: list[tuple[str, Union[bs4.element.Tag, None]]] = _split_text_by_html_data_parts(
        datapoint)
    if seed is not None:
        random.seed(seed)
    for _ in range(num_augmentation_sets):
        augmented_datapoints.append(
            _augment_html_data_once(pieces, 'low', datapoint['note_name']))
        augmented_datapoints.append(
            _augment_html_data_once(pieces, 'mid', datapoint['note_name']))
        augmented_datapoints.append(
            _augment_html_data_once(pieces, 'hi', datapoint['note_name']))
        # augmented_datapoints.append(_augment_html_data_once(pieces, 'high'))
    return augmented_datapoints


def _augment_html_data_once(
        pieces: list[tuple[str, Union[bs4.element.Tag, None]]],
        modification: Literal['low', 'mid', 'high'],
        note_name: str,
        ) -> HTMLData:
    methods = [
        # (push_dollar_signs,0.2),
        (remove_font_styles_at_random, 0.1), (change_font_styles_at_random, 0.2), (change_greek_letters_at_random, 0.1), 
        (remove_math_keywords,0.1), (random_latex_command_removal,0.2),
        (random_word_removal,0.1), (dollar_sign_manipulation,0.05),
        (random_char_modification,0.001)]
    if modification == 'low':
        method_inclusion_chance = 0.3
        scale = 0.5
    elif modification == 'mid':
        method_inclusion_chance = 0.5
        scale = 1.0
    else:
        method_inclusion_chance = 0.8
        scale = 1.5
    
    random_methods = []
    def create_method(method, p, scale):
        return lambda x: method(x, p=p*scale)
    for method, p in methods:
        if random.random() < method_inclusion_chance:
            random_methods.append(create_method(method, p, scale))
    # random_methods = [
    #     lambda x: method(x, p=p*scale) for method, p in methods if random.random() < method_inclusion_chance]
    augmented_pieces = [
        (augment_text(text, random_methods), copy.copy(tag))
        for text, tag in pieces]
    accumulated_len: int = 0
    accumulated_text: str = ""
    tags_with_indices: list[HTMLTagWithIndices] = []
    for text, tag in augmented_pieces:
        accumulated_text = f'{accumulated_text}{text}'
        if tag:
            tag.string = text
            tags_with_indices.append(HTMLTagWithIndices(
                tag, accumulated_len, accumulated_len + len(text)))
        accumulated_len += len(text)
    return HTMLData(note_name=note_name, raw_text=accumulated_text, tags=tags_with_indices)

The `augment_html_data` can be used to augment definition and notation token classification data gathered via `html_data_from_note`.

In [ ]:
mf = MarkdownFile.from_string(
    r"""---
aliases: []
tags: []
---
# Galois group of a separable and normal finite field extension

Let $L/K$ be a separable and normal finite field extension. Its <b definition="Galois group of a separable and normal finite field extension">Galois group</b> <span notation="">$\operatorname{Gal}(L/K)$</span> is...

# Galois group of a separable and normal profinite field extension

In fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that
$L = \varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its **Galois group** **$\operatorname{Gal}(L/K)$**

# See Also
# Meta
## References and Citations
""")

html_data: HTMLData = html_data_from_note(mf)
output = augment_html_data(html_data,seed=None)
html_data, output
# html_data

({'note_name': None,
  'raw_text': 'Let $L/K$ be a separable and normal finite field extension. Its Galois group $\\operatorname{Gal}(L/K)$ is...\n\nIn fact, the notion of a Galois group can be defined for profinite field extensions. Given a separable and normal profinite field extension $L/K$, say that\n$L = \\varinjlim_i L_i$ where $L_i/K$ are finite extensions. Its Galois group $\\operatorname{Gal}(L/K)$\n',
  'tags': [HTMLTagWithIndices(tag=<b definition="Galois group of a separable and normal finite field extension">Galois group</b>, start=64, end=76),
   HTMLTagWithIndices(tag=<span notation="">$\operatorname{Gal}(L/K)$</span>, start=77, end=102),
   HTMLTagWithIndices(tag=<b definition="">Galois group</b>, start=330, end=342),
   HTMLTagWithIndices(tag=<span notation="">$\operatorname{Gal}(L/K)$</span>, start=343, end=368)]},
 [{'note_name': None,
   'raw_text': 'Let $L/K$ be a separable and normal finite field extension. Its Galois group $Gal(L/K)$ is...\n\nIn fact, the notion 

## Gathering data 

The following is sample code to then gather data for definition/notation identification

In [ ]:
#| notest

# TODO: test

notes = [] # Replace with actual notes
vault = '' # Replace with actual vault

# vault = 'C:' # Replace with actual vault
# notes = [] # Replace with actual notes

html_data = [html_data_from_note(note, vault) for note in notes]
max_length = 1022

tokenized_html_data = [tokenize_html_data(html_locus, tokenizer, max_length, def_or_notat_from_html_tag, label2id) for html_locus in html_data]
token_id_data = [token_ids for token_ids, _ in tokenized_html_data]
ner_tag_data = [ner_tag_ids for _, ner_tag_ids in tokenized_html_data]
token_seqs = [token_seq for token_seq in token_ids for token_ids in token_id_data]
ner_tag_seqs = [ner_tag_seq for ner_tag_seq in ner_tag_ids for ner_tag_ids in ner_tag_data]

In [ ]:
#| notest
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
max_length = 1022
label2id = {
    "O": 0,
    "B-definition": 1,
    "I-definition": 2,
    "B-notation": 3,
    "I-notation": 4
} 
id2label = {value: key for key, value in label2id.items()}

In [ ]:
#| notest
note_names, token_seqs, ner_tag_seqs = [], [], []
for html_locus, (token_ids, ner_tag_ids) in zip(html_data, tokenized_html_data):
    note_names.extend([html_locus["Note name"]] * len(token_ids))
    token_seqs.extend(token_ids)
    ner_tag_seqs.extend(ner_tag_ids)

In [ ]:
#| notest
# ner_tags = ClassLabel(names=list(label2id))

# ds = Dataset.from_dict(
#         {"note_name": note_names,
#         "tokens": token_ids,
#         "ner_tags": ner_tag_ids},
#         features=Features(
#             {
#              "note_name": Value(dtype='string'),
#              "tokens": Sequence(Value(dtype='string')),
#              "ner_tags": Sequence(ner_tags)}
#         ))

# ds.save_to_disk(".")

# ds.load_from_disk(".")
    

## Use the trained model

See https://huggingface.co/docs/transformers/tasks/token_classification for training a token classification model.

In [ ]:
# Helper functions

In [ ]:
soup = bs4.BeautifulSoup('', 'html.parser')
tag = soup.new_tag('b', style="border-width:1px;border-style:solid;padding:3px", definition="")
tag.string = 'hi'
tag


<b definition="" style="border-width:1px;border-style:solid;padding:3px">hi</b>

In [ ]:
#| export
def _make_tag(
        text: str,
        entity_type: str # 'definition' or 'notation'
        ) -> bs4.element.Tag:
    """
    Helper function to `_html_tag_data_from_part` and `_consolidate_token_preds`.
    """
    soup = bs4.BeautifulSoup('', 'html.parser')
    if entity_type == 'definition':
        tag = soup.new_tag(
            'b',
            style="border-width:1px;border-style:solid;padding:3px",
            definition="")
    else:
        tag = soup.new_tag(
            'span',
            style="border-width:1px;border-style:solid;padding:3px",
            notation="")
    tag.string = text
    return tag


def _html_tag_data_from_part(
        main_text: str,
        # part: list[dict[str]]) -> tuple[bs4.element.Tag, int, int]:
        part: list[dict[str]] # An item of an output of `_divide_token_preds_into_parts`. Each dict likely contains keys such as `'entity'`, `'score'`, `'index'`, `'word'`, `'start'`, and `'end'`, depending on the model used.
        ) -> tuple[HTMLTagWithIndices]:
    """
    Helper function to `_html_tags_from_token_preds`
    """
    start_token: dict[str] = part[0]
    end_token: dict[str] = part[-1]
    start_char_pos: int = start_token['start']
    end_char_pos: int = end_token['end']

    # Depending on the tokenizer starting spaces might be included in a given token.
    # We exclude such a starting space.
    while main_text[start_char_pos].isspace():
        start_char_pos += 1
    while main_text[end_char_pos-1].isspace():
        end_char_pos -= 1

    # the `'entity'` is either 'I-definition', 'B-definition', 'I-notation',
    # or 'B-notation'
    entity_type = start_token['entity'][2:]
    html_text = main_text[start_char_pos:end_char_pos]
    
    # return (_make_tag(html_text, entity_type), start_char, end_char)
    return HTMLTagWithIndices(_make_tag(html_text, entity_type), start_char_pos, end_char_pos)


In [ ]:
#| hide

main_text = r"Let $I \subset A$ be an ideal. Define its radical by $\sqrt{I}$"

sample_output_1 = _html_tag_data_from_part(
    main_text, [{
        'entity': 'B-definition',
        'score': 0.37319255,
        'index': 25,  # This is moot for the purposes of this test.
        'word': 'radical',
        'start': 42,
        'end': 49
    }])
test_eq(str(sample_output_1.tag), '<b definition="" style="border-width:1px;border-style:solid;padding:3px">radical</b>')

sample_output_2 = _html_tag_data_from_part(
    main_text, [{
        'entity': 'B-notation',
        'score': 0.67021805,
        'index': 27,  # This is moot for the purposes of this test.
        'word': '$',
        'start': 53,
        'end': 54},
    {
        'entity': 'I-notation',
        'score': 0.9748327,
        'index': 28,  # This is moot for the purposes of this test.
        'word': '\\',
        'start': 54,
        'end': 55},
    {
        'entity': 'I-notation',
        'score': 0.9754836,
        'index': 29,  # This is moot for the purposes of this test.
        'word': 'sq',
        'start': 55,
        'end': 57},
    {
        'entity': 'I-notation',
        'score': 0.9750675,
        'index': 30,  # This is moot for the purposes of this test.
        'word': '##rt',
        'start': 57,
        'end': 59},
    {
        'entity': 'I-notation',
        'score': 0.97785944,
        'index': 31,  # This is moot for the purposes of this test.
        'word': '{',
        'start': 59,
        'end': 60},
    {
        'entity': 'I-notation',
        'score': 0.97785944,
        'index': 32,  # This is moot for the purposes of this test.
        'word': 'i',
        'start': 60,
        'end': 61},
    {
        'entity': 'I-notation',
        'score': 0.97785944,
        'index': 33,  # This is moot for the purposes of this test.
        'word': '}',
        'start': 61,
        'end': 62},
    {
        'entity': 'I-notation',
        'score': 0.97785944,
        'index': 34,  # This is moot for the purposes of this test.
        'word': '$',
        'start': 62,
        'end': 63},
    ])
test_eq(str(sample_output_2.tag), r'<span notation="" style="border-width:1px;border-style:solid;padding:3px">$\sqrt{I}$</span>')
# main_text.find('radical')


In [ ]:
#| hide

# With modernbert, the tokenizer tends to capture spaces, within tokens, which creates some subtle issues with spacing.
text = r'Definition 1.1. - We call $\mathscr{U}$-topos, or simply topos if there is no fear of confusion, a category $\mathrm{E}$ such that there is a site $\mathrm{C} \in \mathscr{U}$ such that $\mathrm{E}$ is equivalent to the category $\tilde{\mathrm{C}}$ of $\mathrm{U}$-sheaves of sets over $\mathrm{C}$ .'

part_1 = [
        {'entity': 'B-definition',
        'score': 0.9991014,
        'index': 9,
        'word': 'Ġ$\\',
        'start': 25,
        'end': 28},
        {'entity': 'I-definition',
        'score': 0.99974686,
        'index': 10,
        'word': 'mathscr',
        'start': 28,
        'end': 35},
        {'entity': 'I-definition',
        'score': 0.999652,
        'index': 11,
        'word': '{',
        'start': 35,
        'end': 36},
        {'entity': 'I-definition',
        'score': 0.99985695,
        'index': 12,
        'word': 'U',
        'start': 36,
        'end': 37},
        {'entity': 'I-definition',
        'score': 0.9997553,
        'index': 13,
        'word': '}$-',
        'start': 37,
        'end': 40},
        {'entity': 'I-definition',
        'score': 0.9997918,
        'index': 14,
        'word': 'top',
        'start': 40,
        'end': 43},
        {'entity': 'I-definition',
        'score': 0.9997335,
        'index': 15,
        'word': 'os',
        'start': 43,
        'end': 45},]

part_2 = [
        {'entity': 'B-definition',
        'score': 0.594063,
        'index': 19,
        'word': 'Ġto',
        'start': 56,
        'end': 59},
        {'entity': 'I-definition',
        'score': 0.9056522,
        'index': 20,
        'word': 'pos',
        'start': 59,
        'end': 62}]

sample_output_1: HTMLTagWithIndices = _html_tag_data_from_part(text, part_1)
assert not sample_output_1.tag.text.startswith(' ')
assert sample_output_1.tag.text == '$\\mathscr{U}$-topos'

sample_output_2: HTMLTagWithIndices = _html_tag_data_from_part(text, part_1)
assert not sample_output_2.tag.text.startswith(' ')
assert sample_output_2.tag.text == '$\\mathscr{U}$-topos'

In [ ]:
#| export
def _current_token_continues_the_previous_token(
        current_token: dict, previous_token: dict, note: Optional[VaultNote]
        ) -> bool:
    """
    Helper function to `_divide_token_preds_into_parts`.
    """
    if current_token['entity'].startswith('I-'):
        if current_token['entity'][2:] == previous_token['entity'][2:]:
            return True
        elif note:
            warnings.warn(rf"""
                In the note {note.name} at {note.path()},
                The token '{previous_token['word']}' is marked as '{previous_token['entity']}'
                and the subsequent token '{current_token['word']}' is marked as '{current_token['entity']}',
                which is unusual because the two consecutive tokens seem to be of different
                entities, and yet the latter token does not start with a 'B-'.

                The latter token will be treated like the beginning of a new entity."""
                    )
            return False
    else:
        return False
        

In [ ]:
#| hide
previous_token_1 = {
        'entity': 'I-notation',
        'score': 0.97785944,
        'index': 33,  # This is moot for the purposes of this test.
        'word': '}',
        'start': 61,
        'end': 62}
current_token_1 = {
        'entity': 'I-notation',
        'score': 0.97785944,
        'index': 34,  # This is moot for the purposes of this test.
        'word': '$',
        'start': 62,
        'end': 63}
assert _current_token_continues_the_previous_token(current_token_1, previous_token_1, note=VaultNote('', rel_path='hi'))

# Something like below should hopefully not happen, but it should still give a warning message
previous_token_2 = {
        'entity': 'I-definition',
        'score': 0.97785944,
        'index': 33,  # This is moot for the purposes of this test.
        'word': '}',
        'start': 61,
        'end': 62}
current_token_2 = {
        'entity': 'I-notation',
        'score': 0.97785944,
        'index': 34,  # This is moot for the purposes of this test.
        'word': '$',
        'start': 62,
        'end': 63}

with warnings.catch_warnings(record=True) as w:
    sample_output = _current_token_continues_the_previous_token(current_token_2, previous_token_2, note=VaultNote('', rel_path='hi'))
    assert w
    assert not sample_output

previous_token_3 = {
        'entity': 'I-definition',
        'score': 0.97785944,
        'index': 33,  # This is moot for the purposes of this test.
        'word': '##tion',
        'start': 58,
        'end': 62}
current_token_3 = {
        'entity': 'B-notation',
        'score': 0.97785944,
        'index': 34,  # This is moot for the purposes of this test.
        'word': '$',
        'start': 62,
        'end': 63}

assert not _current_token_continues_the_previous_token(current_token_3, previous_token_3, note=VaultNote('', rel_path='hi'))



In [ ]:
#| export
def _divide_token_preds_into_parts(
        token_preds: list[dict[str]],
        note: VaultNote,
        excessive_space_threshold: int
        ) -> list[list[dict[str]]]:
    """
    Divide `token_preds` into parts so that each part
    represents a single definition/notation marking.

    Helper function to `_html_tags_from_token_preds`.
    """
    token_preds_parts = []
    for current_token in token_preds:
        if not token_preds_parts:
            token_preds_parts.append([current_token])
            continue
        prev_token = token_preds_parts[-1][-1]
        if _current_token_continues_the_previous_token(
                current_token, prev_token, note):
            prev_token_end = prev_token['end']
            cur_token_start = current_token['start']
            if prev_token_end + excessive_space_threshold >= cur_token_start and note:
                Warning(rf"""
                    In the note {note.name} at {note.path()},
                    There seems to be excessive space between the token
                    {prev_token['word']} and {current_token['word']}, which
                    seem to be part of the same entity"""
                        )
            token_preds_parts[-1].append(current_token)
        else:
            token_preds_parts.append([current_token])
    return token_preds_parts


In [ ]:
#| hide

main_text = r"Let $I \subset A$ be an ideal. Define its radical by $\sqrt{I}$"

preds = [
    {
        'entity': 'B-definition',
        'score': 0.37319255,
        'index': 25,  # This is moot for the purposes of this test.
        'word': 'radical',
        'start': 42,
        'end': 49
    },
    {
        'entity': 'B-notation',
        'score': 0.67021805,
        'index': 27,  # This is moot for the purposes of this test.
        'word': '$',
        'start': 53,
        'end': 54},
    {
        'entity': 'I-notation',
        'score': 0.9748327,
        'index': 28,  # This is moot for the purposes of this test.
        'word': '\\',
        'start': 54,
        'end': 55},
    {
        'entity': 'I-notation',
        'score': 0.9754836,
        'index': 29,  # This is moot for the purposes of this test.
        'word': 'sq',
        'start': 55,
        'end': 57},
    {
        'entity': 'I-notation',
        'score': 0.9750675,
        'index': 30,  # This is moot for the purposes of this test.
        'word': '##rt',
        'start': 57,
        'end': 59},
    {
        'entity': 'I-notation',
        'score': 0.97785944,
        'index': 31,  # This is moot for the purposes of this test.
        'word': '{',
        'start': 59,
        'end': 60},
    {
        'entity': 'I-notation',
        'score': 0.97785944,
        'index': 32,  # This is moot for the purposes of this test.
        'word': 'i',
        'start': 60,
        'end': 61},
    {
        'entity': 'I-notation',
        'score': 0.97785944,
        'index': 33,  # This is moot for the purposes of this test.
        'word': '}',
        'start': 61,
        'end': 62},
    {
        'entity': 'I-notation',
        'score': 0.97785944,
        'index': 34,  # This is moot for the purposes of this test.
        'word': '$',
        'start': 62,
        'end': 63},
    ]

output = _divide_token_preds_into_parts(
    preds, VaultNote('', rel_path='hi'), excessive_space_threshold=2
)

# Test that the list finds two parts, one for the definition, and the other for the notation.
test_eq(len(output), 2)
test_eq(len(output[0]), 1)
test_eq(len(output[1]), len(preds) - 1)


In [ ]:
#| export
def _ranges_overlap(
        current_1: tuple[HTMLTagWithIndices],
        current_2: tuple[HTMLTagWithIndices]
        # current_1: tuple[bs4.element.Tag, int, int],
        # current_2: tuple[bs4.element.Tag, int, int]
        ) -> bool:
    """
    Based on https://stackoverflow.com/a/64745177

    Helper function to `_collate_html_tags`, `_consolidate_token_preds`.
    """
    return max(current_1.start, current_2.start) < min(current_1.end, current_2.end)


In [ ]:
#| hide
# In actuality, there should be bs4.element.Tag objects in place of ''.
assert _ranges_overlap(HTMLTagWithIndices('', 3, 8), HTMLTagWithIndices('', 6, 12))
assert _ranges_overlap(HTMLTagWithIndices('', 3, 8), HTMLTagWithIndices('', 3, 4))
assert not _ranges_overlap(HTMLTagWithIndices('', 3, 8), HTMLTagWithIndices('', 8, 9))
assert _ranges_overlap(HTMLTagWithIndices('', 6, 12), HTMLTagWithIndices('', 3, 8))

In [ ]:
#| export
# If the ML model predicts 
# predictions made around 
latex_commands_to_avoid = [
    # Document structure
    "\\documentclass", "\\usepackage", "\\begin", "\\end",
    # Sectioning
    "\\part", "\\chapter", "\\section", "\\subsection", "\\subsubsection", "\\paragraph", "\\subparagraph",
    # Referencing and citation
    "\\ref", "\\pageref", "\\cite", "\\bibitem", "\\bibliography", "\\bibliographystyle",
    
    # Lists
    "\\enumerate", "\\itemize", "\\item", "\\description",
    
    # Footnotes
    "\\footnote", "\\footnotemark", "\\footnotetext",
    
    # Floats and captions
    # "\\figure", "\\table", "\\caption", "\\centering",
    
    # Page formatting
    "\\newpage", "\\clearpage", "\\cleardoublepage", "\\pagebreak", "\\newline", "\\linebreak",
    
    # Definitions and counters
    "\\newcommand", "\\renewcommand", "\\def", "\\let", "\\setcounter", "\\addtocounter", "\\stepcounter", "\\refstepcounter",
    
    # Index and glossary
    "\\index", "\\glossary", "\\printindex", "\\printglossary",
    
    # Hyperlinks (if using hyperref package)
    "\\href", "\\url", "\\hypersetup", "\\hyperlink", "\\hypertarget",
    
    # Spacing
    "\\hspace", "\\vspace", "\\phantom", "\\vphantom", "\\hphantom",
    
    # Alignment environments
    "\\center", "\\flushleft", "\\flushright",
    
    # Verbatim and code
    "\\verb", "\\verbatim", "\\lstlisting",
    
    # Theorems and proofs
    "\\theorem", "\\lemma", "\\proof", "\\corollary",
    
    # Cross-referencing
    "\\cref", "\\Cref", "\\vref", "\\Vref",  # from cleveref package
    
    # Table of contents related
    "\\tableofcontents", "\\listoffigures", "\\listoftables",
    
    # Appendix
    "\\appendix",
    
    # Author and title information
    "\\title", "\\author", "\\date", "\\maketitle",
    
    # Abstract
    "\\abstract", "\\keywords",
    
    # Including external files
    "\\input", "\\include",
    
    # Page numbering
    "\\pagenumbering", "\\thepage",
    
    # Margin adjustments
    "\\marginpar", "\\marginparsep", "\\marginparwidth",
    
    # Columns
    "\\twocolumn", "\\onecolumn", "\\multicolumn",
]


In [ ]:
#| export
def _str_contains_latex_command_to_avoid(text):
    """
    Helper function to `_consolidate_token_preds`
    """
    # Check if any command from the list is in the text
    for command in latex_commands_to_avoid:
        if command.lower() in text:
            return True
    return False

In [ ]:
#| hide
test_strings = [
    r"This is a normal sentence.",
    r"This sentence contains \section{A Section}.",
    r"Here's a \footnote{This is a footnote.}",
    r"Let's \begin{enumerate} some items.",
    r"\href{https://example.com}{This is a hyperlink}",
]

for i, string in enumerate(test_strings, 1):
    result = _str_contains_latex_command_to_avoid(string)
    print(f"String {i}: {'Contains' if result else 'Does not contain'} a LaTeX command to avoid.")

String 1: Does not contain a LaTeX command to avoid.
String 2: Contains a LaTeX command to avoid.
String 3: Contains a LaTeX command to avoid.
String 4: Contains a LaTeX command to avoid.
String 5: Contains a LaTeX command to avoid.


In [ ]:
#| export
def _consolidate_token_preds(
        main_text: str,
        tag_data: list[HTMLTagWithIndices]
        ) -> list[HTMLTagWithIndices]:
    """
    Since the model's predictions can yield some odd results
    (e.g. notations not being marked for an entire LaTeX string
    $<span notation="">$S_k := ...</span>$$), this function tries
    to consolidate some oddities.
    
    """
    latex_inds = latex_indices(main_text)
    extended_tag_data = _extend_tag_data_ranges(main_text, latex_inds, tag_data)
    # extended_tag_data = _extend_tag_data_ranges_to_encompass_latex(main_text, latex_inds, tag_data)
    # extended_tag_data = _extend_tag_data_ranges_to_border_spaces(main_text, extended_tag_data)
    tag_data_notats_chopped = _cutoff_notation_tag_data(main_text, extended_tag_data)
    # Go through the extended tag data to throw out overlapping ones.
    ultimate_tag_data: list[HTMLTagWithIndices] = []
    for tag_point in tag_data_notats_chopped:
        if (_no_overlap_with_previous_tag_data(ultimate_tag_data, tag_point)
                and not _str_contains_latex_command_to_avoid(tag_point.tag.text)):
            ultimate_tag_data.append(tag_point)
    return ultimate_tag_data


def _extend_tag_data_ranges(
        main_text: str,
        latex_inds: list[tuple[int, int]],
        tag_data: list[HTMLTagWithIndices],
        ) -> list[HTMLTagWithIndices]:
    """
    Extend tag data so that
    1. the tag data does not start or end within any latex math mode string.
    2. the tag data is immediately preceded by whitespace (or the start/end of line)
       and followed by whitespace or punctuation
    3. the tag data does not start/end in the middle of the arguments of a latex command.

    Helper function to `_consolidate_token_preds`.
    """
    # TODO: make sure that tag has balanced curly braces 
    extended_tag_data = []
    for tag_tuple in tag_data:
        tag_tuple_before_extension = tag_tuple
        while True:
            tag_tuple_after_extension = _extend_tag_data_range_for_math_mode(
                tag_tuple_before_extension, main_text, latex_inds)
            tag_tuple_after_extension = _extend_tag_data_range_to_border_space_or_punc(
                tag_tuple_after_extension, main_text)
            tag_tuple_after_extension = _extend_tag_data_ranges_to_balance_curly_braces(
                tag_tuple_after_extension, main_text)
            tag_tuple_before_extension = tag_tuple_after_extension 
            if (tag_tuple_before_extension[1] == tag_tuple_after_extension[1]
                    and tag_tuple_before_extension[2] == tag_tuple_after_extension[2]):
                break
        extended_tag_data.append(tag_tuple_after_extension)
    return extended_tag_data


def _update_tag_data(
        tag_tuple: HTMLTagWithIndices,
        main_text: str,
        new_start: int,
        new_end: int) -> HTMLTagWithIndices:
    new_text = main_text[new_start:new_end]
    tag_type = 'definition' if 'definition' in tag_tuple.tag.attrs else 'notation'
    new_tag = _make_tag(new_text, tag_type)
    return HTMLTagWithIndices(new_tag, new_start, new_end)


def _extend_tag_data_range_for_math_mode(
        tag_tuple: HTMLTagWithIndices,
        main_text: str,
        latex_inds: list[tuple[int, int]],
        ) -> HTMLTagWithIndices:
    """
    Extend tag data so that the tag data does not start or end within latex math mode string.
    """
    extended_range = [tag_tuple.start, tag_tuple.end]
    for tex_range in latex_inds:
        if not _ranges_overlap(HTMLTagWithIndices(0, tex_range[0], tex_range[1]), tag_tuple):
            continue
        extended_range = (min(extended_range[0], tex_range[0]), max(extended_range[1], tex_range[1]))
    return _update_tag_data(tag_tuple, main_text, extended_range[0], extended_range[1])


def _extend_tag_data_range_to_border_space_or_punc(
        tag_tuple: HTMLTagWithIndices,
        main_text: str,
        ) -> HTMLTagWithIndices:
    """
    Extend tag data so that the tag data borders spaces or punctuations.

    Helper function to `_consolidate_token_preds`.
    """
    combined_range = [tag_tuple.start, tag_tuple.end]
    while combined_range[0] != 0 and not main_text[combined_range[0]-1].isspace():
        combined_range[0] -= 1
    # while combined_range[1] != len(main_text) and not main_text[combined_range[1]].isspace():
    while combined_range[1] != len(main_text) and is_not_space_and_not_punc(main_text[combined_range[1]]):
        combined_range[1] += 1
    return _update_tag_data(tag_tuple, main_text, combined_range[0], combined_range[1])


def _extend_tag_data_ranges_to_balance_curly_braces(
        tag_tuple: HTMLTagWithIndices,
        main_text: str
        ) -> list[HTMLTagWithIndices]:
    """
    Extend tag data to balance curly braces
    """
    combined_range = [tag_tuple.start, tag_tuple.end]
    while not _is_balanced_braces(main_text[combined_range[0]:combined_range[1]]):
        changed = False
        while combined_range[0] != 0 and _first_curly_bracket(main_text[combined_range[0]:combined_range[1]]) == r'}':
            combined_range[0] -= 1
            changed = True
        while combined_range[1] != len(main_text) and _last_curly_bracket(main_text[combined_range[0]:combined_range[1]]) == r'{':
            combined_range[1] += 1
            changed = True
        if not changed:
            break
    return _update_tag_data(tag_tuple, main_text, combined_range[0], combined_range[1])


def _cutoff_notation_tag_data(
        main_text: str,
        tag_data: list[HTMLTagWithIndices],
        ) -> list[HTMLTagWithIndices]:
    """
    Helper function to `_consolidate_token_preds`.

    Guarantees that a notation tag is a pure math mode latex string
    by cutting only the pure math mode string
    that occurs within it. Assumes that `_extend_tag_data_ranges_to_encompass_latex`
    works as intended.
    """
    cutout_notation_tag_data: list[HTMLTagWithIndices] = []
    for tag, start, end in tag_data:
        if not 'notation' in tag.attrs:
            cutout_notation_tag_data.append(HTMLTagWithIndices(tag, start, end))
            continue
        tag_text = main_text[start:end]
        tex_inds_in_tagged = latex_indices(tag_text)
        for sub_start, sub_end in tex_inds_in_tagged:
            tex_str = main_text[start+sub_start:start+sub_end]
            cutout_tag = _make_tag(tex_str, 'notation')
            cutout_notation_tag_data.append(
                HTMLTagWithIndices(cutout_tag, start+sub_start, start+sub_end))
    return cutout_notation_tag_data



def _no_overlap_with_previous_tag_data(
        ultimate_tag_data: list[HTMLTagWithIndices],
        current_tag_data: HTMLTagWithIndices  # Current tag data
        ) -> bool:
    """
    Return `True`, if the `current_tag_data` does not overlap with
    any tag data that will be ultimately added. 

    Helper function for `_consolidate_token_preds`.
    """
    for prev in reversed(ultimate_tag_data):
        if _ranges_overlap(prev, current_tag_data):
            return False
    return True
    

In [ ]:
#| hide
main_text = 'Hi. This is some text. Here is a notation $$M_k :=... $$ and here is some more $$G_k :=...$$'

soup = bs4.BeautifulSoup('', 'html.parser')
# In actuality, the tag data will have more information, but the following
# is good enough
# for the purposes of this test
tag_1 = soup.new_tag('span', notation='')
tag_2 = soup.new_tag('span', notation='')
# The following tries to test an erroneous short/incomplete marking of the very first 
# dollar sign `$` as well as the subtext 'G_K ' of the second math mode text.

tag_data = [HTMLTagWithIndices(tag_1, 42, 43), HTMLTagWithIndices(tag_2, 81, 85)]
output = _consolidate_token_preds(main_text, tag_data)
test_eq(main_text[output[0][1]: output[0][2]], '$$M_k :=... $$')
test_eq(main_text[output[1][1]: output[1][2]], '$$G_k :=...$$')


# In the following example, the notation is a priori
# found to be 'zeta(s)$ as $$\zeta'. 
# So first, the tagged data is extended to encompass
# '$\zeta(s)$ as $$\zeta(s) = ...$$' and then pure latex
# math mode str are extracted
main_text = r'Define $\zeta(s)$ as $$\zeta(s) = ...$$  '

soup = bs4.BeautifulSoup('', 'html.parser')
tag_1 = soup.new_tag('span', notation='')
tag_data = [HTMLTagWithIndices(tag_1,9,27)]
output = _consolidate_token_preds(main_text, tag_data)
# main_text.find('zeta')
test_eq(main_text[output[0][1]:output[0][2]], r'$\zeta(s)$')
test_eq(main_text[output[1][1]:output[1][2]], r'$$\zeta(s) = ...$$')



# In the following example, we have erroneous
# notation and definition markings which start/end within the same 
# math mode string.
# In this case, the preceding marking takes precedence and 
# the latter overlapping marking is discarded;
# this is an unfortunate feature that must be implemented to get
# around shortcomings of the model.
main_text = r'The Riemann zeta function $\zeta(s)$ is defined as...'
soup = bs4.BeautifulSoup('', 'html.parser')
tag_1 = soup.new_tag('b', definition='')
tag_2 = soup.new_tag('span', notation='')
tag_data = [HTMLTagWithIndices(tag_1,4,27), HTMLTagWithIndices(tag_2,26,30)] # the tag predictions overlap at $ \zeta(s)$.
output = _consolidate_token_preds(main_text, tag_data)
test_eq(main_text[output[0][1]:output[0][2]], r'Riemann zeta function $\zeta(s)$')

# In the following example, the definition tag prediction ends within the argument
# of the \emph command. 
main_text = r'The \emph{arc complex}'
soup = bs4.BeautifulSoup('', 'html.parser')
tag = soup.new_tag('b', definition="")
tag_data = [HTMLTagWithIndices(tag,4,21)]
output = _consolidate_token_preds(main_text, tag_data)
test_eq(main_text[output[0][1]:output[0][2]], r'\emph{arc complex}')

# In the following example, the definition tag prediction ends amidst of the \emph command. 
main_text = r'The \emph{arc complex}'
soup = bs4.BeautifulSoup('', 'html.parser')
tag = soup.new_tag('b', definition="")
tag_data = [HTMLTagWithIndices(tag,6,8)]
output = _consolidate_token_preds(main_text, tag_data)
test_eq(main_text[output[0][1]:output[0][2]], r'\emph{arc complex}')

# In the following example, a bad command gets predicted. 
# The consolidation should thus leave out this prediction.
main_text = r'\section{lalalala} We define a homotopy'
soup = bs4.BeautifulSoup('', 'html.parser')
tag = soup.new_tag('b', definition="")
tag_data = [HTMLTagWithIndices(tag,0,2)]
output = _consolidate_token_preds(main_text, tag_data)
test_eq(len(output), 0)
# print(output)

# In the following example, the definition tag prediction ends with a punctuation mark
# (a period in this case). 
main_text = r'This is called a homotopy. There exist...'
soup = bs4.BeautifulSoup('', 'html.parser')
tag = soup.new_tag('b', definition="")
tag_data = [HTMLTagWithIndices(tag,17,24)]
output = _consolidate_token_preds(main_text, tag_data)
test_eq(main_text[output[0][1]:output[0][2]], r'homotopy')

# The `_extend_tag_data_range_to_border_space_or_punc` function should not try to
# encapsulate the period as part of the definition tag.
tag_data = [HTMLTagWithIndices(tag,18,23)]
output = _consolidate_token_preds(main_text, tag_data)
test_eq(main_text[output[0][1]:output[0][2]], r'homotopy')


In [ ]:
#| export
def _html_tags_from_token_preds(
        main_text: str,
        token_preds: list[dict[str]], # An output of `pipeline(text)`; Each dict likely contains keys such as `'entity'`, `'score'`, `'index'`, `'word'`, `'start'`, and `'end'`, depending on the model used.
        note: VaultNote,
        excessive_space_threshold: int
        ) -> list[HTMLTagWithIndices]:  # Tag element, start, end, where main_text[start:end] needs to be replaced by the tag element.
        # ) -> list[tuple[bs4.element.Tag, int, int]]:  # Tag element, start, end, where main_text[start:end] needs to be replaced by the tag element.
    """
    Return HTML tags for definition and notation classification.

    Helper function to `auto_mark_def_and_notats`.
    """
    parts: list[list[dict[str]]] = _divide_token_preds_into_parts(
        token_preds, note, excessive_space_threshold)
    return [_html_tag_data_from_part(main_text, part) for part in parts]


In [ ]:
#| hide
# TODO: test _html_tags_from_token_preds

In [ ]:
#| export
def _collate_html_tags(
        tag_data_1: list[HTMLTagWithIndices],
        tag_data_2: list[HTMLTagWithIndices],
    ) -> list[tuple[bs4.element.Tag], int, int]:
    """
    Collates the lists of HTML tags and the indices within a certain text
    (which is not-needed for this function and hence not included)
    that the HTML tags need to replace.

    If there are entries in `tag_data_1` and `tag_data_2` with overlapping
    ranges, then the entry from `tag_data_1` is prioritized and the entry
    from `tag_data_2` is discarded.

    Helper function to `auto_mark_def_and_notats`
    """
    collated_list: list[HTMLTagWithIndices] = []
    i, j = 0, 0
    while i < len(tag_data_1) and j < len(tag_data_2):
        current_1 = tag_data_1[i]
        current_2 = tag_data_2[j]
        if _ranges_overlap(current_1, current_2): # Ignore current_2
            j += 1
            continue
        if current_1[1] > current_2[1]:
            collated_list.append(current_2)
            j += 1
        else:
            collated_list.append(current_1)
            i += 1
    while i < len(tag_data_1):
        collated_list.append(tag_data_1[i])
        i += 1
    while j < len(tag_data_2):
        collated_list.append(tag_data_2[j])
        j += 1
    return collated_list




In [ ]:
#| hide


tag_data_1 = [
    HTMLTagWithIndices('', 0, 1),
    HTMLTagWithIndices('', 9, 12),
    HTMLTagWithIndices('', 20, 21)
]

tag_data_2 = [
    HTMLTagWithIndices('', 2, 4),
    HTMLTagWithIndices('', 6, 7),
    HTMLTagWithIndices('', 8, 10), # This should be discarded
    HTMLTagWithIndices('', 10, 13), # This should be discarded
    HTMLTagWithIndices('', 17, 20),
    HTMLTagWithIndices('', 21, 24)
]
output = _collate_html_tags(tag_data_1, tag_data_2)
test_eq(output, [
    HTMLTagWithIndices('', 0, 1), HTMLTagWithIndices('', 2, 4), HTMLTagWithIndices('', 6, 7),
    HTMLTagWithIndices('', 9, 12), HTMLTagWithIndices('', 17, 20), HTMLTagWithIndices('', 20, 21), 
    HTMLTagWithIndices('', 21, 24)])

In [ ]:
#| export
def _add_nice_boxing_attrs_to_def_and_notat_tags(
        html_tag_data: list[HTMLTagWithIndices]
        ) -> list[HTMLTagWithIndices]:
    """
    Add HTML tag attributes to draw boxes around notation data

    Helper function to `auto_mark_def_and_notats`.
    """
    listy: list[HTMLTagWithIndices] = []
    for tag, start, end in html_tag_data:
        if ('notation' in tag.attrs or 'definition' in tag.attrs) and 'style' not in tag.attrs:
            tag.attrs['style'] = "border-width:1px;border-style:solid;padding:3px"
        listy.append((tag, start, end)) 
    return listy



In [ ]:
#| hide
soup = bs4.BeautifulSoup('', "html.parser")
tag = soup.new_tag("span", notation="")
tag.string = 'hi'
tag_data = [
    HTMLTagWithIndices(tag, 0, 2),
]
output = _add_nice_boxing_attrs_to_def_and_notat_tags(tag_data)
assert "style" in output[0][0].attrs

tag = soup.new_tag("span", definition="")
tag.string = 'hi'
tag_data = [
    HTMLTagWithIndices(tag, 0, 2),
]
output = _add_nice_boxing_attrs_to_def_and_notat_tags(tag_data)
assert "style" in output[0][0].attrs

In [ ]:
#| export
def def_and_notat_preds_by_model(
        text: str,  
        pipeline # The pipeline object created using the token classification model and its tokenizer
        ) -> list[HTMLTagWithIndices]: # HTMLTAgWithIndices consists of an HTML tag carrying the data of the prediction and ints marking where in `text` the definition or notation is at.
    """
    Predict where definitions and notations occur in `text`

    This function uses some of the same helper functions as
    `auto_mark_def_and_notats`, but does not raise warning messages as
    in `auto_mark_def_and_notats`.
    """
    tag_data = _html_tags_from_token_preds(text, pipeline(text), None, 2)
    return tag_data

In [ ]:
# TODO: test

In [ ]:
#| export
def auto_mark_def_and_notats(
        note: VaultNote,  # The standard information note in which to find the definitions and notations.
        pipeline: pipelines.token_classification.TokenClassificationPipeline, # The token classification pipeline that is used to predict whether tokens are part of definitions or notations introduced in the text.
        # remove_existing_def_and_notat_markings: bool = False,  # If `True`, remove definition and notation markings (both via surrounding by double asterisks `**` as per the legacy method and via HTML tags)
        excessive_space_threshold: int = 2,
        add_boxing_attr_to_existing_def_and_notat_markings: bool = True # If `True`, then nice attributes are added to the existing notation HTML tags, if not already present.
    ) -> None:
    """
    Predict and mark where definitions and notation occur in a note using
    a token classification ML model.

    Assumes that the note is a standard information note that does not
    have a lot of "user modifications", such as footnotes, links,
    and HTML tags. If
    there are many modifications, then these might be deleted.

    Assumes that the paragraphs in the text of the note are "not too long".
    Currently, this means that the paragraphs in the number of tokens
    in the text of the note should (roughly) not exceed 
    `pipeline.tokenizer.model_max_length`.

    Existing markings for definition and notation data (i.e. by
    surrounding with double asterisks or by HTML tags) are preserved
    (and turned into HTML tags), unless the markings overlap with 
    predictions, in which case the original is preserved (and still
    turned into an HTML tag if possible)

    Since the model can make "invalid" predictions (mostly those which
    start or end within a LaTeX math mode str), the actual markings
    are not necessarily direct translates from the model's predictions.
    See the helper function `_consolidate_token_preds` for more details
    on how this is implemented.
    
    **Raises**
    Warning messages (`UserWarning`) are printed in the following situations:

    - There are two consecutive tokens within the `pipeline`'s predictions
      of different entity types (e.g. one is predicted to belong within a
      definition and the other within a notation), but the latter token's
      predicted `'entity'` more specifically begins with `'I-'` (i.e. is
      `'I-definition'` or `'I-notation'`) as opposed to `'B-'`.
        - `note`'s name, and path are included in the warning message in
          this case.
    - There are two consecutive tokens within the `pipeline`'s predictions
      which the pipeline predicts to belong to the same entity, and yet
      there is excessive space (specified by `excessive_space_threshold`)
      between the end of the first token and the start of the second.

    """
    mf = MarkdownFile.from_vault_note(note)
    _process_mf(mf)
    first_non_metadata_line, see_also_line = _get_main_text_lines(mf)
     
    main_text = mf.text_of_lines(first_non_metadata_line, see_also_line)
    main_text = _format_main_text_and_add_html_tag_data(
        note, pipeline, add_boxing_attr_to_existing_def_and_notat_markings,
        excessive_space_threshold, main_text)
    _write_text_with_html_tag_preds_to_note(
        note, mf, main_text, first_non_metadata_line, see_also_line)


def _process_mf(
        mf: MarkdownFile) -> None:
    """
    Merge display math mode as needed

    Helper function to `auto_mark_def_and_notats`
    """
    mf.merge_display_math_mode()
    mf.merge_display_math_mode_into_preceding_text()


def _get_main_text_lines(
        mf: MarkdownFile) -> tuple[int, int]:
    """Helper function to `auto_mark_def_and_notats`"""
    tuppy = mf.metadata_lines()
    if tuppy is not None:
        first_non_metadata_line = tuppy[1] + 1
    else:
        first_non_metadata_line = 0 
    see_also_line = mf.get_line_number_of_heading('See Also')
    return first_non_metadata_line, see_also_line


def _format_main_text_and_add_html_tag_data(
        note: VaultNote,
        pipeline: pipelines.token_classification.TokenClassificationPipeline, # The token classification pipeline that is used to predict whether tokens are part of definitions or notations introduced in the text.
        add_boxing_attr_to_existing_def_and_notat_markings: bool,
        excessive_space_threshold: int,
        main_text: str,  # The main text to format and to add HTML tag data to
        ) -> str:
    """Helper function to `auto_mark_def_and_notats`"""
    main_text = add_space_to_lt_symbols_without_space(main_text)
    main_text = convert_double_asterisks_to_html_tags(main_text)
    main_text, existing_html_tag_data = remove_html_tags_in_text(main_text)
    if add_boxing_attr_to_existing_def_and_notat_markings:
        existing_html_tag_data = _add_nice_boxing_attrs_to_def_and_notat_tags(
            existing_html_tag_data)

    html_tags_to_add = _get_token_preds_by_dividing_main_text(
        main_text, pipeline, note, excessive_space_threshold)

    html_tags_to_add_back = _collate_html_tags(
        existing_html_tag_data, html_tags_to_add)
    return add_HTML_tag_data_to_raw_text(main_text, html_tags_to_add_back)


def _get_token_preds_by_dividing_main_text(
        main_text: str,
        pipeline: pipelines.token_classification.TokenClassificationPipeline, # The token classification pipeline that is used to predict whether tokens are part of definitions or notations introduced in the text. Here, the tokenizer of this pipeline is used to estimate how many tokens a piece of subtext will have.
        note: VaultNote,
        excessive_space_threshold: int,    
        # ) -> list[tuple[bs4.element.Tag, int, int]]:  # Tag element, start, end, where main_text[start:end] needs to be replaced by the tag element.
        ) -> list[HTMLTagWithIndices]:  # Tag element, start, end, where main_text[start:end] needs to be replaced by the tag element.
    """
    Divide the `main_text` into not-too-long pieces to return HTML tag predictions

    Helper function for `_format_main_text_and_add_html_tag_data`.
    """
    pieces_start_and_end = _divide_main_text(main_text, pipeline)
    cumulative_html_tags_in_main = []
    for start_of_piece, end_of_piece in pieces_start_and_end:
        # text = main_text[start_of_piece:end_of_piece]
        text = main_text[start_of_piece:]
        html_tags_in_piece: list[HTMLTagWithIndices] = _html_tags_from_token_preds(
            text, pipeline(text), note, excessive_space_threshold)
        html_tags_in_piece = _consolidate_token_preds(
            text, html_tags_in_piece)
        # start and end indices need to be re-adjusted with respect to their places in `main_text`
        html_tags_for_piece_in_main_text = [
            (tag, start_of_piece + start, start_of_piece + end)
            for tag, start, end in html_tags_in_piece]
        cumulative_html_tags_in_main = _collate_html_tags(
            cumulative_html_tags_in_main, html_tags_for_piece_in_main_text)
    return cumulative_html_tags_in_main


def _divide_main_text(
        main_text: str,
        pipeline: pipelines.token_classification.TokenClassificationPipeline, # The token classification pipeline that is used to predict whether tokens are part of definitions or notations introduced in the text. Here, the tokenizer of this pipeline is used to estimate how many tokens a piece of subtext will have.
        # ) -> list[tuple[str, int, int]]:  # The str is a chunk of text, the first int is the index in `main_text` that the chunk starts at, and the second int is the approximate token length of the text. Appending all the chunks of text as they are should result back in the original text.
        ) -> list[tuple[int, int]]:  # Each tuple is a start and end range for pieces of `main_text` to be considered for predictions
    """Divides `main_text` so that predictions can be made on smaller chunks of text.
    
    Assumes that dividing `main_text` along newline characters `\n` will result in
    pieces that are "not too long".

    Helper function to `_format_main_text_and_add_html_tag_data`.


    """
    main_text.split('\n')
    tokenizer = pipeline.tokenizer
    newline_indices = [i for i, char in enumerate(main_text) if char == '\n']
    newline_indices.insert(0, 0)
    chunks = []  # list[tuple[str, int, int]]  # The str is a chunk of text, the first int is the index in `main_text` that the chunk starts at, and the second int is the approximate token length of the text. Appending all the chunks of text as they are should result back in the original text.
    for start, end in pairwise(newline_indices):
        chunk = main_text[start:end]
        chunks.append((chunk, start, len(tokenizer(chunk)['input_ids'])))
    last_chunk = main_text[newline_indices[-1]:]
    chunks.append((last_chunk, newline_indices[-1], len(tokenizer(chunk))))
    return _find_places_to_divide_from_chunks(chunks, pipeline)


def _find_places_to_divide_from_chunks(
        chunks: list[tuple[str, int, int]], # The str is a chunk of text, the first int is the index in `main_text` that the chunk starts at, and the second int is the approximate token length of the text. Appending all the chunks of text as they are should result back in the original text.
        pipeline: pipelines.token_classification.TokenClassificationPipeline, # The token classification pipeline that is used to predict whether tokens are part of definitions or notations introduced in the text. Here, the tokenizer of this pipeline is used to estimate how many tokens a piece of subtext will have.
        ) -> list[tuple[int, int]]: # Each tuple is a start and end range for pieces of `main_text` to be considered for predictions
    """Identify appropriate indices in `main_text` where (overlapping)
    pieces in `main_text` should start/end for predictions with `pipeline`.
    
    Helper function to `_divide_main_text`.

    We describe how this function is implemented: starting at the first chunk
    (chunks are non-overlapping), start to consider consecutive chunks to
    make up a piece. So maybe we have chunks

        A B C D E F ....

    We build a piece chunk-by-chunk, considering the total token length of the
    built sub-piece along the way. The first chunk within a sub-piece 
    that makes the sub-piece of token-length greater than half the max
    token length with respect to `pipeline.tokenizer` will become the start of the
    next piece, unless the very first chunk in the piece is already longer than half the max
    token length with respect to the tokenizer (this is to ensure that the
    piece-building process does not keep starting at the same chunk).
    Moreover, a piece will stop building as soon as its token-length exceeds
    the max length of the tokenizer.

    For instance, maybe the max length (`tokenizer.model_max_length`)
    for the tokenizer is 512, and the chunks
    are of the following length:

        A   B    C  D   E    F     ...
        76  130  70 13  150  140   ...

    We first build the piece starting at A:

        A
        76

    We continue building the piece by "appending" B:

        A   B
        76  130

    Once we append C as well, the piece's length is now 276 and hence over half of 512,
    so the next piece will start at C: 

        A   B    C
        76  130  70

    Subsequently, we continue building the piece. Only once F is appended does the 
    length of the entire piece exceed 512 (the length is 579):

        A   B    C  D   E    F
        76  130  70 13  150  140
    
    And then we begin building the next piece from C.

    Also, consider an example where the first chunk's length exceeds half the max length
    of the tokenizer:

        A   B    C   ...
        300 200  100 ...

    Here, the first piece will consist of the chunks A, B, and C because
    the length of the piece exceeds the max length of 512 only after appending C.
    To guarantee that the next piece does not start with the chunk A again, B is 
    used as the first chunk in the next piece:

        B    C   ...
        200  100 ...

    If any chunk's token length exceeds the tokenizer's max_model_length, then
    the pipeline/model can only predict on the starting tokens in the chunk. 
    As such, the chunks must not be "too long" for best results on the model's predictions.
    """
    tokenizer = pipeline.tokenizer
    start_chunk_index, next_piece_start_chunk_index = 0, 0
    current_piece_token_len = 0
    pieces_start_and_end = []
    i = 0
    while i < len(chunks):
        chunk = chunks[i]
        current_piece_token_len = current_piece_token_len + chunk[2]
        if (current_piece_token_len > tokenizer.model_max_length / 2
                and start_chunk_index == next_piece_start_chunk_index):
            # Mark where the next piece should start
            next_piece_start_chunk_index = i if start_chunk_index != i else i+1
        if (current_piece_token_len > tokenizer.model_max_length):
            _append_to_pieces_start_and_end(
                pieces_start_and_end, chunks[start_chunk_index], chunk)
            i, start_chunk_index = (
                next_piece_start_chunk_index, next_piece_start_chunk_index)
            current_piece_token_len = 0
            continue
        i += 1
    # Add the last chunk at the end
    _append_to_pieces_start_and_end(
        pieces_start_and_end, chunks[start_chunk_index], chunks[-1])
    return pieces_start_and_end


def _append_to_pieces_start_and_end(
        pieces_start_and_end: list[tuple[int, int]],
        start_chunk: tuple[str, int, int],
        end_chunk: tuple[str, int, int]
        ) -> None:
    """Helper function to `_find_places_to_divide_from_chunks`"""
    start_char_index = start_chunk[1]
    end_char_index = end_chunk[1] + len(end_chunk[0])
    pieces_start_and_end.append([start_char_index, end_char_index])



def _write_text_with_html_tag_preds_to_note(
        note: VaultNote,
        mf: MarkdownFile,
        main_text: str,
        first_non_metadata_line: int,
        see_also_line: int
        ) -> None:
    """
    Final step of the marking process; the new contents of the note are written.

    Helper function to `auto_mark_def_and_notats`
    """
    mf.remove_lines(first_non_metadata_line, see_also_line)
    mf.insert_line(first_non_metadata_line,
                   {'type': MarkdownLineEnum.DEFAULT, 'line': main_text})
    mf.add_tags('_auto/def_and_notat_identified')
    mf.write(note)


In the following examples, we mock pipeline objects instead of using actual ones.

In the below example, we run the `auto_mark_def_and_notats` function on a note that has double asterisks `**` surrounding parts of the text that introduced definitions or notations. In these cases, appropriate HTML tags replace the double asterisks instead.

In [ ]:
with (tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir,
      mock.patch('__main__.pipelines.token_classification.TokenClassificationPipeline') as mock_pipeline):
    temp_vault = Path(temp_dir) / 'test_vault_6'
    shutil.copytree(_test_directory() / 'test_vault_6', temp_vault)

    mock_pipeline.tokenizer.model_max_length = 512

    vn = VaultNote(temp_vault, name='reference_with_tag_labels_Definition 2')
    print("Text before:\n\n")
    print(vn.text())
    print("\n\n\nText after:\n")
    auto_mark_def_and_notats(vn, mock_pipeline)
    print(vn.text())
    mf = MarkdownFile.from_vault_note(vn)
    assert mf.has_tag('_auto/def_and_notat_identified')




Text before:


---
cssclass: clean-embeds
aliases: []
tags: [_meta/literature_note, _meta/definition, _meta/notation]
---
# Ring of integers modulo $n$[^1]

Let $n \geq 1$ be an integer. The **ring of integers modulo $n$**, denoted by **$\mathbb{Z}/n\mathbb{Z}$**, is, informally, the ring whose elements are represented by the integers with the understanding that $0$ and $n$ are equal.

More precisely, $\mathbb{Z}/n\mathbb{Z}$ has the elements $0,1,\ldots,n-1$.

...


# See Also
- [[reference_with_tag_labels_Exercise 1|reference_with_tag_labels_Z_nZ_is_a_ring]]
# Meta
## References

## Citations and Footnotes
[^1]: Kim, Definition 2



Text after:

---
cssclass: clean-embeds
aliases: []
tags: [_meta/notation, _meta/definition, _auto/def_and_notat_identified, _meta/literature_note]
---
# Ring of integers modulo $n$[^1]

Let $n \geq 1$ be an integer. The <b definition="" style="border-width:1px;border-style:solid;padding:3px">ring of integers modulo $n$</b>, denoted by <span notation="" s

In [ ]:
# TODO: more examples with pipeline mocking actual outputs